## Setup

In [2]:
import os
import numpy as np
import time
import cv2
from __future__ import print_function

## Define parameters

In [221]:
USE_CAM = 1
DISP_FULL_SCREEN = 1
DISP_BW = False
LOOP = True    # loop through videos
INCREMENT = False
DECREMENT = False

BORDER_STYLE = 0
HEIGHT_MULT = 0.5
WIDTH_MULT = 0.5
MULT_INC = 0.05

THRESH = False
THRESH_STYLE = 0
USE_ALL_CHAN = True
USE_R_CHAN = False
USE_G_CHAN = False
USE_B_CHAN = False

GRAD_STYLE = 0

FFT = False
FFT_OFFSET = 5
FFT_OFFSET_INC = 1
FFT_R_CH_OFFSET = 0
FFT_G_CH_OFFSET = 0
FFT_B_CH_OFFSET = 0
FFT_CH_OFFSET_INC = 5
FFT_B_CH_STYLE = 0
FFT_G_CH_STYLE = 0
FFT_R_CH_STYLE = 0

KEY_PRESS_ZERO = False
KEY_PRESS_ONE = False
KEY_PRESS_TWO = False
KEY_PRESS_THREE = False
KEY_PRESS_MINUS = False
KEY_PRESS_PLUS = False
KEY_PRESS_RESET = False

## Loop

In [230]:
# select video source
if USE_CAM:
    cap = cv2.VideoCapture(0)
else:
    cap = cv2.VideoCapture('/media/data/Dropbox/Git/vid-viz/data/test.mp4')
    
# get video properties
target_fps = cap.get(cv2.CAP_PROP_FPS)
tot_fr_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

fr_count = 0
while(cap.isOpened()):

    time_pre = time.time()
    
    ret, frame = cap.read()
    
    if USE_CAM: #DISP_FULL_SCREEN
        # crop image to have same aspect ratio as monitor
        frame = frame[60:-60,:,:]
    
    [im_height, im_width, im_channels] = frame.shape

    # use keyboard input to change effect
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('s'):
        # toggle color
        DISP_BW = not DISP_BW
    elif key == ord('z'):
        BORDER_STYLE = (BORDER_STYLE + 1) % 3
    elif key == ord('R'):
        INCREMENT = True
    elif key == ord('T'):
        DECREMENT = True
    elif key == ord('t'):
        THRESH = not THRESH
    elif key == ord('y'):
        THRESH_STYLE = (THRESH_STYLE + 1) % 2
    elif key == ord('r'):
        # for thresholding
        USE_ALL_CHAN = False
        USE_R_CHAN = True
        USE_G_CHAN = False
        USE_B_CHAN = False
    elif key == ord('g'):
        # for thresholding
        USE_ALL_CHAN = False
        USE_R_CHAN = False
        USE_G_CHAN = True
        USE_B_CHAN = False
    elif key == ord('b'):
        # for thresholding
        USE_ALL_CHAN = False
        USE_R_CHAN = False
        USE_G_CHAN = False
        USE_B_CHAN = True
    elif key == ord('v'):
        USE_ALL_CHAN = True
        USE_R_CHAN = False
        USE_G_CHAN = False
        USE_B_CHAN = False
    elif key == ord('m'):
        GRAD_STYLE = (GRAD_STYLE + 1) % 2    
    elif key == ord('f'):
        FFT = not FFT
        # turn off other effects for now
#         THRESH = False
#         GRAD_STYLE = 0

    elif key == ord('0'):
        KEY_PRESS_ZERO = True
    elif key == ord('1'):
        KEY_PRESS_ONE = True
    elif key == ord('2'):
        KEY_PRESS_TWO = True
    elif key == ord('3'):
        KEY_PRESS_THREE = True
    elif key == ord('-'):
        KEY_PRESS_MINUS = True
    elif key == ord('='):
        KEY_PRESS_PLUS = True
    elif key == ord('/'):
        KEY_PRESS_RESET = True
        
    """ implement effects """
    
#     if DISP_BW:
#         frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
#     tx = fr_count % im_height
#     ty = fr_count % im_height
#     M = np.float32([[1, 0, tx], [0, 1, ty]])
#     frame = cv2.warpAffine(frame, M, (im_width, im_height))

    
        
    if THRESH:
        
        # BINARY | BINARY_INV | TRUNC | TOZERO | TOZERO_INV
        if THRESH_STYLE == 0:
            THRESH_TYPE = cv2.THRESH_BINARY
        elif THRESH_STYLE == 1:
            THRESH_TYPE = cv2.THRESH_BINARY_INV
            
        # MEAN_C | GAUSSIAN_C
        ADAPTIVE_THRESH_TYPE = cv2.ADAPTIVE_THRESH_MEAN_C
        THRESH_CEIL = 255
        THRESH_BLOCK = 21
        THRESH_C = 4
        
        if USE_ALL_CHAN:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            gray_frame = cv2.adaptiveThreshold(gray_frame, THRESH_CEIL, 
                                          ADAPTIVE_THRESH_TYPE, THRESH_TYPE, 
                                          THRESH_BLOCK, THRESH_C)
            frame[:,:,0] = gray_frame
            frame[:,:,1] = gray_frame
            frame[:,:,2] = gray_frame
            frame = cv2.medianBlur(frame, 11)
        elif USE_R_CHAN:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # zero out blue channel, threshold red channel
            frame[:,:,0] = 0
            frame[:,:,2] = cv2.adaptiveThreshold(gray_frame, THRESH_CEIL, 
                                                 ADAPTIVE_THRESH_TYPE, THRESH_TYPE, 
                                                 THRESH_BLOCK, THRESH_C)
            frame = cv2.medianBlur(frame, 11)
        elif USE_G_CHAN:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # zero out red channel, threshold green channel
            frame[:,:,2] = 0
            frame[:,:,1] = cv2.adaptiveThreshold(gray_frame, THRESH_CEIL,
                                                 ADAPTIVE_THRESH_TYPE, THRESH_TYPE,
                                                 THRESH_BLOCK, THRESH_C)
            frame = cv2.medianBlur(frame, 11)
        elif USE_B_CHAN:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # zero out green channel, threshold red channel
            frame[:,:,1] = 0
            frame[:,:,0] = cv2.adaptiveThreshold(gray_frame, THRESH_CEIL,
                                                 ADAPTIVE_THRESH_TYPE, THRESH_TYPE,
                                                 THRESH_BLOCK, THRESH_C)
            frame = cv2.medianBlur(frame, 11)
    
#     if GRAD_STYLE == 1:
#         grad = cv2.Sobel(frame, cv2.CV_16S, 1, 0, ksize=7)
#     elif GRAD_STYLE == 2:
#         grad = cv2.Sobel(frame, cv2.CV_16S, 0, 1, ksize=7)
#     elif GRAD_STYLE == 3:
#         grad = cv2.Scharr(frame, cv2.CV_16S, 1, 0)
#     elif GRAD_STYLE == 4:
#         grad = cv2.Scharr(frame, cv2.CV_16S, 0, 1)
#     elif GRAD_STYLE == 5:
#         grad = cv2.Laplacian(frame, cv2.CV_16S, ksize=7)
#     elif GRAD_STYLE == 6:
#         grad = cv2.Canny(frame, 100, 200)   

    if GRAD_STYLE > 0:
        grad = cv2.Canny(frame, 100, 200)  
        if USE_R_CHAN:
            frame[:,:,:2] = 0
            frame[:,:,2] = grad
        elif USE_G_CHAN:
            frame[:,:,[0,2]] = 0
            frame[:,:,1] = grad
        elif USE_B_CHAN:
            frame[:,:,1:3] = 0
            frame[:,:,0] = grad
        elif USE_ALL_CHAN:
            frame[:,:,0] = grad
            frame[:,:,1] = grad
            frame[:,:,2] = grad
            
    if FFT:
        
        # update high-pass filter window size
        if INCREMENT:
            FFT_OFFSET += FFT_OFFSET_INC
            INCREMENT = False
        if DECREMENT:
            FFT_OFFSET -= FFT_OFFSET_INC
            if FFT_OFFSET < 0:
                FFT_OFFSET = 0
            DECREMENT = False
        if KEY_PRESS_RESET:
            FFT_R_CH_OFFSET = 0
            FFT_G_CH_OFFSET = 0
            FFT_B_CH_OFFSET = 0
            KEY_PRESS_RESET = False
            
        """ Calculate FFT """
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # pad image to optimize fft calculations
        orows, ocols = frame_gray.shape
        nrows = cv2.getOptimalDFTSize(orows)
        ncols = cv2.getOptimalDFTSize(ocols)
        frame_gray = cv2.copyMakeBorder(frame_gray, 0, nrows-orows, 0, ncols-ocols, 
                                        cv2.BORDER_CONSTANT, value=0)
        rows, cols = frame_gray.shape # update shape
        
        # take fft
        fshift = np.fft.fftshift(cv2.dft(np.float32(frame_gray), flags = cv2.DFT_COMPLEX_OUTPUT))
        
        # filter
        crow, ccol = int(rows/2), int(cols/2)
        fshift[crow-FFT_OFFSET:crow+FFT_OFFSET, ccol-FFT_OFFSET:ccol+FFT_OFFSET] = 0
        frame_back = cv2.idft(np.fft.ifftshift(fshift))
        frame_back = cv2.magnitude(frame_back[:,:,0], frame_back[:,:,1])
        frame_back = frame_back[:orows, :ocols] # update shape to original
        frame_back = (frame_back - np.min(frame_back)) / (np.max(frame_back) - np.min(frame_back)) * 255
        
        """ determine how to use resulting image """
        if USE_R_CHAN:
            
            if KEY_PRESS_MINUS:
                FFT_R_CH_OFFSET = (FFT_R_CH_OFFSET - FFT_CH_OFFSET_INC) % 255
                KEY_PRESS_MINUS = False
            if KEY_PRESS_PLUS:
                FFT_R_CH_OFFSET = (FFT_R_CH_OFFSET + FFT_CH_OFFSET_INC) % 255
                KEY_PRESS_PLUS = False
                
            if KEY_PRESS_ZERO:
                FFT_R_CH_STYLE = 0 # gray
                KEY_PRESS_ZERO = False
            if KEY_PRESS_ONE:
                FFT_R_CH_STYLE = 1 # offset
                KEY_PRESS_ONE = False
            if KEY_PRESS_TWO:
                FFT_R_CH_STYLE = 2 # offset + ifft
                KEY_PRESS_TWO = False
            if KEY_PRESS_THREE:
                FFT_R_CH_STYLE = 3 # offset - ifft
                KEY_PRESS_THREE = False
                
        elif USE_G_CHAN:
            
            if KEY_PRESS_MINUS:
                FFT_G_CH_OFFSET = (FFT_G_CH_OFFSET - FFT_CH_OFFSET_INC) % 255
                KEY_PRESS_MINUS = False
            if KEY_PRESS_PLUS:
                FFT_G_CH_OFFSET = (FFT_G_CH_OFFSET + FFT_CH_OFFSET_INC) % 255
                KEY_PRESS_PLUS = False
                
            if KEY_PRESS_ZERO:
                FFT_G_CH_STYLE = 0 # gray
                KEY_PRESS_ZERO = False
            if KEY_PRESS_ONE:
                FFT_G_CH_STYLE = 1 # offset
                KEY_PRESS_ONE = False
            if KEY_PRESS_TWO:
                FFT_G_CH_STYLE = 2 # offset + ifft
                KEY_PRESS_TWO = False
            if KEY_PRESS_THREE:
                FFT_G_CH_STYLE = 3 # offset - ifft
                KEY_PRESS_THREE = False
                
        elif USE_B_CHAN:

            if KEY_PRESS_MINUS:
                FFT_B_CH_OFFSET = (FFT_B_CH_OFFSET - FFT_CH_OFFSET_INC) % 255
                KEY_PRESS_MINUS = False
            if KEY_PRESS_PLUS:
                FFT_B_CH_OFFSET = (FFT_B_CH_OFFSET + FFT_CH_OFFSET_INC) % 255
                KEY_PRESS_PLUS = False
                
            if KEY_PRESS_ZERO:
                FFT_B_CH_STYLE = 0 # gray
                KEY_PRESS_ZERO = False
            if KEY_PRESS_ONE:
                FFT_B_CH_STYLE = 1 # offset
                KEY_PRESS_ONE = False
            if KEY_PRESS_TWO:
                FFT_B_CH_STYLE = 2 # offset + ifft
                KEY_PRESS_TWO = False
            if KEY_PRESS_THREE:
                FFT_B_CH_STYLE = 3 # offset - ifft
                KEY_PRESS_THREE = False
        
        """ update frame """
        frame = np.zeros(shape=frame.shape, dtype='uint8')
        
        if FFT_R_CH_STYLE == 0:
            frame[:,:,2] = frame_gray
        elif FFT_R_CH_STYLE == 1:
            frame[:,:,2] = FFT_R_CH_OFFSET
        elif FFT_R_CH_STYLE == 2:
            frame[:,:,2] = FFT_R_CH_OFFSET + frame_back.astype(np.uint8)
        elif FFT_R_CH_STYLE == 3:
            frame[:,:,2] = FFT_R_CH_OFFSET - frame_back.astype(np.uint8)
            
        if FFT_G_CH_STYLE == 0:
            frame[:,:,1] = frame_gray
        elif FFT_G_CH_STYLE == 1:
            frame[:,:,1] = FFT_G_CH_OFFSET
        elif FFT_G_CH_STYLE == 2:
            frame[:,:,1] = FFT_G_CH_OFFSET + frame_back.astype(np.uint8)
        elif FFT_G_CH_STYLE == 3:
            frame[:,:,1] = FFT_G_CH_OFFSET - frame_back.astype(np.uint8)
            
        if FFT_B_CH_STYLE == 0:
            frame[:,:,0] = frame_gray
        elif FFT_B_CH_STYLE == 1:
            frame[:,:,0] = FFT_B_CH_OFFSET
        elif FFT_B_CH_STYLE == 2:
            frame[:,:,0] = FFT_B_CH_OFFSET + frame_back.astype(np.uint8)
        elif FFT_B_CH_STYLE == 3:
            frame[:,:,0] = FFT_B_CH_OFFSET - frame_back.astype(np.uint8)
            
        frame = cv2.medianBlur(frame, 9)
    
    if BORDER_STYLE > 0:
    
        if INCREMENT:
            HEIGHT_MULT += MULT_INC
            WIDTH_MULT += MULT_INC
            INCREMENT = False
        if DECREMENT:
            HEIGHT_MULT -= MULT_INC
            if HEIGHT_MULT < 0:
                HEIGHT_MULT = 0
            WIDTH_MULT -= MULT_INC
            if WIDTH_MULT < 0:
                WIDTH_MULT = 0
            DECREMENT = False
        
        if BORDER_STYLE == 1:

            # top, bottom, left, right
            frame = cv2.copyMakeBorder(frame, int(im_height*HEIGHT_MULT), int(im_height*HEIGHT_MULT),
                                              int(im_width*WIDTH_MULT), int(im_width*WIDTH_MULT),
                                              cv2.BORDER_WRAP)
        elif BORDER_STYLE == 2:
            # top, bottom, left, right
            frame = cv2.copyMakeBorder(frame, int(im_height*2*HEIGHT_MULT), 0,
                                              int(im_width*2*WIDTH_MULT), 0,
                                              cv2.BORDER_REFLECT)
        
    # display image
    if DISP_FULL_SCREEN:
        cv2.namedWindow('frame', cv2.WND_PROP_FULLSCREEN)
        cv2.setWindowProperty('frame', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
        cv2.imshow('frame', frame)
    else:
        cv2.imshow('frame', frame)
        if fr_count == 0:
            cv2.moveWindow('frame', 0, 0)
    
    # reset video
    fr_count += 1
    if fr_count == tot_fr_count and LOOP:
        # reset frame postion to zero
        cap.set(2,1)
        fr_count = 1
    
    # calculate, limit and output fps
    time_post = time.time()
    time_tot = time_post - time_pre
    if time_tot < 1/target_fps:
        time.sleep(1/target_fps - time_tot)
    time_post = time.time()
    time_tot = time_post - time_pre
    fps = 1.0 / time_tot
    print('\r%03i fps' % fps, end='')
    
    
cap.release()
cv2.destroyWindow('frame')

015 fps

### Good FFT settings
#### Green and black
* B_CH_STYLE = 1, B_CH_OFFSET = 0
* G_CH_STYLE = 2, G_CH_OFFSET = 0
* R_CH_STYLE = 2, R_CH_OFFSET = 0
* FFT_OFFSET = 4
* THRESHOLDING OFF

#### Green and red
* B_CH_STYLE = 1, B_CH_OFFSET = 0
* G_CH_STYLE = 2, G_CH_OFFSET = 0
* R_CH_STYLE = 3, R_CH_OFFSET = 0
* FFT_OFFSET = 1
* THRESHOLDING ON

#### Green and purple
* B_CH_STYLE = 0, B_CH_OFFSET = 0
* G_CH_STYLE = 2, G_CH_OFFSET = 0
* R_CH_STYLE = 2, R_CH_OFFSET = 0
* FFT_OFFSET = 2
* THRESHOLDING ON

In [229]:
whos

Variable               Type                   Data/Info
-------------------------------------------------------
ADAPTIVE_THRESH_TYPE   int                    0
BORDER_STYLE           int                    1
DECREMENT              bool                   False
DISP_BW                bool                   True
DISP_FULL_SCREEN       int                    1
FFT                    bool                   True
FFT_B_CHAN_STYLE       int                    0
FFT_B_CH_OFFSET        int                    0
FFT_B_CH_STYLE         int                    0
FFT_CH_OFFSET_INC      int                    5
FFT_G_CHAN_STYLE       int                    0
FFT_G_CH_OFFSET        int                    0
FFT_G_CH_STYLE         int                    2
FFT_OFFSET             int                    2
FFT_OFFSET_INC         int                    1
FFT_R_CHAN_STYLE       int                    0
FFT_R_CH_OFFSET        int                    0
FFT_R_CH_STYLE         int                    2
GRAD_STYLE    

In [74]:
whos

Variable               Type                   Data/Info
-------------------------------------------------------
ADAPTIVE_THRESH_TYPE   int                    0
BORDER_STYLE           int                    2
DISP_BW                bool                   False
DISP_FULL_SCREEN       int                    1
GRAD_STYLE             int                    0
HEIGHT_MULT            float                  1.6
LOOP                   bool                   True
MULT_INC               float                  0.05
THRESH                 bool                   True
THRESH_BLOCK           int                    21
THRESH_C               int                    4
THRESH_CEIL            int                    255
THRESH_STYLE           int                    0
THRESH_TYPE            int                    0
USE_ALL_CHAN           bool                   False
USE_B_CHAN             bool                   False
USE_CAM                int                    1
USE_G_CHAN             bool                   

## FFT play

In [186]:
from matplotlib import pyplot as plt

img = cv2.imread('/media/data/Dropbox/Git/vid-viz/data/test2.png')
scale = 1
img = cv2.resize(img, None, fx=scale, fy=scale)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# pad image to optimize fft calculations
orows, ocols = img_gray.shape
nrows = cv2.getOptimalDFTSize(orows)
ncols = cv2.getOptimalDFTSize(ocols)
img_gray = cv2.copyMakeBorder(img_gray, 0, nrows-orows, 0, ncols-ocols, cv2.BORDER_CONSTANT, value=0)
rows, cols = img_gray.shape
crow, ccol = int(rows/2), int(cols/2)
offset = 5

fshift = np.fft.fftshift(cv2.dft(np.float32(img_gray), flags = cv2.DFT_COMPLEX_OUTPUT))
magnitude_spectrum = 20*np.log(cv2.magnitude(fshift[:,:,0], fshift[:,:,1]))

fshift[crow-offset:crow+offset, ccol-offset:ccol+offset] = 0
magnitude_spectrum2 = 20*np.log(cv2.magnitude(fshift[:,:,0], fshift[:,:,1]))
img_back = cv2.idft(np.fft.ifftshift(fshift))
img_back = cv2.magnitude(img_back[:,:,0], img_back[:,:,1])
img_back = img_back[:orows, :ocols]
img_gray = img_gray[:orows, :ocols]
img_back = (img_back - np.min(img_back)) / (np.max(img_back) - np.min(img_back)) * 255

# plt.subplot(141)
# plt.imshow(img)
# plt.title('Input image')
# plt.xticks([]), plt.yticks([])

# plt.subplot(142)
# plt.imshow(magnitude_spectrum, cmap='gray')
# plt.title('Magnitude Spectrum I')
# plt.xticks([]), plt.yticks([])

# plt.subplot(143)
# plt.imshow(magnitude_spectrum2, cmap='gray')
# plt.title('Magnitude Spectrum II')
# plt.xticks([]), plt.yticks([])

# plt.subplot(144)
# plt.imshow(img_back, cmap='gray')
# plt.title('Filtered Image')
# plt.xticks([]), plt.yticks([])

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:22: RuntimeWarning: divide by zero encountered in log


In [191]:
img_new = np.zeros(shape=img.shape, dtype='uint8')
img_new[:,:,0] = img_gray #[:,:,0]
img_new[:,:,1] = 255 - img_back.astype(np.uint8) #img[:,:,1]
img_new[:,:,2] = 255 #img[:,:,2] #img_gray

plt.imshow(img_new)
plt.title('Filtered Image')
plt.xticks([]), plt.yticks([])

plt.show()

In [82]:
img = cv2.imread('/media/data/Dropbox/Git/vid-viz/data/test2.jpg')
scale = 1
img = cv2.resize(img, None, fx=scale, fy=scale)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.xticks([]), plt.yticks([])

plt.show()

In [157]:
# img_back = (img_back - np.min(img_back)) / (np.max(img_back) - np.min(img_back)) * 255

# img_back = img_back.astype(np.uint8)

# img_back2 = cv2.medianBlur(img_back, 5)
_, img_back2 = cv2.threshold(img_back, 20, 255, cv2.THRESH_BINARY)
plt.imshow(255 - img_back2, cmap='gray')
plt.xticks([]), plt.yticks([])

plt.show()